In [ ]:
import torch
import numpy as np

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Etude 1
Let $u$ be a vector of $2D$ points. We need to get a matrix $A$ whose element $(i,j)$ depends on euclidean distance between $u_i$ and $u_j$.

$$u \in \mathbb{R}^{N \times 2}$$
$$A \in \mathbb{R}^{N \times N}, \ A_{ij} = f(\|u_i - u_j\|) $$

Particularly, let's take $f(x) = exp(-\frac{x}{4})$.

### Naive approach

In [ ]:
def naive(u, N):
    result = torch.zeros(N, N)
    for i in range(N):
        for j in range(N):
            result[i,j] = torch.exp(-0.25 * torch.dist(u[i], u[j]))
    return result

### Vectorized approaches

In [ ]:
def fully_vectorized(u, N):
    u_ = u.reshape(1, *u.shape) # add one dimension for transposing
    A_ = u_.permute((1, 0, 2)) - u_ # row - column: broadcasting subtraction
    return A_.pow_(2).sum(dim=2).sqrt_().mul_(-0.25).exp_()

In [ ]:
def fully_vectorized_gpu(u, N):
    u.to(device)
    u_ = u.reshape(1, *u.shape)
    A_ = u_.permute((1, 0, 2)) - u_
    return A_.pow_(2).sum(dim=2).sqrt_().mul_(-0.25).exp_()

In [ ]:
def one_dim_vectorized(u, N):
    result = torch.zeros(N, N)
    for i in range(N):
        du = u[i] - u
        result[i,:] = du.pow_(2).sum(dim=1).sqrt_().mul_(-0.25).exp_()
    return result

### Comparison

#### fully vectorized vs. naive

In [ ]:
N = 10**3
u = torch.randn(N, 2)

In [ ]:
%%time
A = naive(u, N)

In [ ]:
%%time
B = fully_vectorized(u, N)

In [ ]:
assert torch.equal(A, B)

#### 1-dim vectorized vs. fully vectorized

In [ ]:
N = 10**4
u = torch.randn(N, 2)

In [ ]:
%%timeit
A = fully_vectorized(u, N)

In [ ]:
%%timeit
B = one_dim_vectorized(u, N)

In [ ]:
assert torch.equal(A, B)

#### cpu vs. gpu

In [ ]:
N = 10**4
u = torch.randn(N, 2)
v = torch.randn(N, 2, device=device)

In [ ]:
%%timeit
A = fully_vectorized(u, N)

In [ ]:
%%timeit
B = fully_vectorized_gpu(v, N)

In [ ]:
torch.equal(A, B)

### t(N)

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

def bench(func, n_range, repeat=3):
    t = []
    for n in n_range:
        t0 = []
        for i in range(repeat):
            u = torch.randn(n, 2)
            start = time.time()
            func(u, n)
            end = time.time()
            t0.append(end-start)
        t.append(sum(t0)/len(t0))
    return t

In [ ]:
n = list(range(10**3, 6*10**3, 10**3))
t_fv = bench(func=fully_vectorized, n_range=n)
t_1dv = bench(func=one_dim_vectorized, n_range=n)

In [ ]:
plt.plot(n, t_1dv, label='1dv')
plt.plot(n, t_fv, label='fv')
plt.ylabel('t, sec')
plt.xlabel('N')
plt.legend()
plt.savefig("1d_vs_fully")